In [1]:
!git clone https://github.com/gauravgola96/data.git

Cloning into 'data'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 32320 (delta 2), reused 42 (delta 2), pack-reused 32277
Receiving objects: 100% (32320/32320), 594.23 MiB | 27.80 MiB/s, done.
Resolving deltas: 100% (1464/1464), done.
Checking out files: 100% (47316/47316), done.


In [1]:
import os
import keras
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import Model
from keras.utils import multi_gpu_model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dropout,GlobalAveragePooling2D,Dense
from keras import applications as apps
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.models import Model,Sequential
from keras.layers import Dense,Flatten
from keras.layers import BatchNormalization,Activation


Using TensorFlow backend.


In [2]:
# !pip install keras==2.2.4
keras.__version__

'2.2.4'

In [3]:
os.listdir('/content/data/Augmented_Data')

['new_approach(Training_with_data_subsets)',
 'train_crop',
 'test_crop',
 'validation_crop']

In [4]:
print(os.listdir('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)'))

train_path = '/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/train'

subset_1 = '/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/subset_1'

subset_2 = '/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/subset_2'

subset_3 = '/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/subset_3'

val_path = '/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/test_files'

['test_files', 'model_checkpoint', 'subset_2', 'subset_3', 'subset_1', 'train', 'model_subset']


* Performed Data augmentation on Colourblock class
* Each class has around 7% of data 
* Classes are balanced

In [5]:
count = []
for i in list(os.walk(top='/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/train'))[0][1]:
    counts_images = len(os.listdir('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/train/'+i)) 
    print('Number of Images of class %s'%i,'\t','Percentage %f'%(counts_images/5760 *100),'\t','counts %d'%counts_images)
    count.append(len(os.listdir('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/train/'+i)))
print('Total Count %d'%sum(count))

Number of Images of class Patterned 	 Percentage 8.333333 	 counts 480
Number of Images of class Checked 	 Percentage 8.333333 	 counts 480
Number of Images of class striped 	 Percentage 8.333333 	 counts 480
Number of Images of class typography 	 Percentage 8.333333 	 counts 480
Number of Images of class abstract 	 Percentage 8.333333 	 counts 480
Number of Images of class polka dots 	 Percentage 8.333333 	 counts 480
Number of Images of class floral 	 Percentage 8.333333 	 counts 480
Number of Images of class Colourblock 	 Percentage 8.333333 	 counts 480
Number of Images of class solid 	 Percentage 8.333333 	 counts 480
Number of Images of class Printed 	 Percentage 8.333333 	 counts 480
Number of Images of class Melange 	 Percentage 8.333333 	 counts 480
Number of Images of class graphic 	 Percentage 8.333333 	 counts 480
Total Count 5760


* Performing transfer learning method on Xception deep learning model.
* Saving weights from subset_1 data and loading same weights on same network.
* Re-running the same network on subset_2 data and saving its weights.
* Same procedure for subset_3 data.


In [0]:
# Loading model.
model_Xcep = keras.applications.xception.Xception(weights='imagenet',include_top=False,input_shape=(299,299,3))

In [0]:
# Freezing first 100 layers
# Around 25% of the parameters are trainable.
for layers in model_Xcep.layers[:50]:
    layers.trainable=False

In [0]:
# Customizing network for our dataset(12 classes)
x = model_Xcep.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='sigmoid')(x)
x = Dropout(0.5)(x)
pred = Dense(12,activation='softmax')(x)
model = Model(inputs=model_Xcep.input, outputs=pred)


In [0]:
nadam=keras.optimizers.nadam(lr =0.001)
model.compile(optimizer=nadam,loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
# os.makedirs('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_subset/')
# os.makedirs('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_checkpoint/')

In [0]:

filepath='/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_checkpoint/patterntest.{epoch:02d}-{val_loss:.2f}.hdf5'

# tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

mcCallBack = keras.callbacks.ModelCheckpoint(filepath,save_best_only=True,mode='auto',monitor='val_acc')

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20)

decaylearn = keras.callbacks.ReduceLROnPlateau(patience=5)

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    vertical_flip=False,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2)

train_generator= train_datagen.flow_from_directory(subset_1,batch_size=64,target_size=(299,299),class_mode='categorical')

Found 353 images belonging to 12 classes.


In [13]:
model.fit_generator(train_generator,
                  epochs=10,steps_per_epoch = train_generator.samples//64)

Epoch 1/10
5/5 [==============================] - 27s 5s/step - loss: 2.5942 - acc: 0.1406
Epoch 2/10
5/5 [==============================] - 14s 3s/step - loss: 1.9892 - acc: 0.3621
Epoch 3/10
5/5 [==============================] - 11s 2s/step - loss: 1.2528 - acc: 0.5887
Epoch 4/10
5/5 [==============================] - 11s 2s/step - loss: 1.0179 - acc: 0.6690
Epoch 5/10
5/5 [==============================] - 11s 2s/step - loss: 0.6808 - acc: 0.7712
Epoch 6/10
5/5 [==============================] - 11s 2s/step - loss: 0.4584 - acc: 0.8397
Epoch 7/10
5/5 [==============================] - 13s 3s/step - loss: 0.4702 - acc: 0.8500
Epoch 8/10
5/5 [==============================] - 11s 2s/step - loss: 0.2659 - acc: 0.9273
Epoch 9/10
5/5 [==============================] - 11s 2s/step - loss: 0.1681 - acc: 0.9594
Epoch 10/10
5/5 [==============================] - 11s 2s/step - loss: 0.2735 - acc: 0.9114


In [0]:
model.save_weights('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_subset/subset_1_wts.h5')

* For subset_2 data


In [0]:

# Loading model with no weights
model_Xcep = keras.applications.xception.Xception(weights=None,include_top=False,input_shape=(299,299,3))

In [0]:
# Freezing first 100 layers
# Around 25% of the parameters are trainable.
for layers in model_Xcep.layers[:80]:
    layers.trainable=False

In [0]:
# Customizing network for our dataset(12 classes)
x = model_Xcep.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='sigmoid')(x)
x = Dropout(0.5)(x)
pred = Dense(12,activation='softmax')(x)
model = Model(inputs=model_Xcep.input, outputs=pred)

In [0]:
model.load_weights('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_subset/subset_1_wts.h5')

In [0]:
nadam=keras.optimizers.nadam(lr =0.001)
model.compile(optimizer=nadam,loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
train_generator= train_datagen.flow_from_directory(subset_2,batch_size=64,target_size=(299,299),class_mode='categorical')

Found 720 images belonging to 12 classes.


In [21]:
model.fit_generator(train_generator,
                  epochs=15,steps_per_epoch = train_generator.samples//64)

Epoch 1/15
11/11 [==============================] - 31s 3s/step - loss: 1.7519 - acc: 0.4858
Epoch 2/15
11/11 [==============================] - 24s 2s/step - loss: 1.1228 - acc: 0.6037
Epoch 3/15
11/11 [==============================] - 25s 2s/step - loss: 0.7766 - acc: 0.7343
Epoch 4/15
11/11 [==============================] - 24s 2s/step - loss: 0.6850 - acc: 0.7722
Epoch 5/15
11/11 [==============================] - 23s 2s/step - loss: 0.4678 - acc: 0.8508
Epoch 6/15
11/11 [==============================] - 24s 2s/step - loss: 0.4337 - acc: 0.8511
Epoch 7/15
11/11 [==============================] - 24s 2s/step - loss: 0.3779 - acc: 0.8772
Epoch 8/15
11/11 [==============================] - 23s 2s/step - loss: 0.2529 - acc: 0.9183
Epoch 9/15
11/11 [==============================] - 23s 2s/step - loss: 0.2865 - acc: 0.8958
Epoch 10/15
11/11 [==============================] - 23s 2s/step - loss: 0.2349 - acc: 0.9357
Epoch 11/15
11/11 [==============================] - 23s 2s/step - lo

In [0]:
model.save_weights('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_subset/subset__wts.h5')

* For subset_3 data

In [0]:
# Loading model with no weights
model_Xcep = keras.applications.xception.Xception(weights=None,include_top=False,input_shape=(299,299,3))

# Freezing first 100 layers
# Around 25% of the parameters are trainable.
for layers in model_Xcep.layers[:100]:
    layers.trainable=False

# Customizing network for our dataset(12 classes)
x = model_Xcep.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='sigmoid')(x)
x = Dropout(0.5)(x)
pred = Dense(12,activation='softmax')(x)
model = Model(inputs=model_Xcep.input, outputs=pred)

In [0]:
model.load_weights('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_subset/subset__wts.h5')

In [0]:
nadam=keras.optimizers.nadam(lr =0.001)
model.compile(optimizer=nadam,loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
train_generator= train_datagen.flow_from_directory(subset_3,batch_size=64,target_size=(299,299),class_mode='categorical')

Found 720 images belonging to 12 classes.


In [27]:
model.fit_generator(train_generator,
                  epochs=15,steps_per_epoch = train_generator.samples//64)


Epoch 1/15
11/11 [==============================] - 28s 3s/step - loss: 1.7013 - acc: 0.5597
Epoch 2/15
11/11 [==============================] - 21s 2s/step - loss: 0.9945 - acc: 0.6804
Epoch 3/15
11/11 [==============================] - 22s 2s/step - loss: 0.7215 - acc: 0.7590
Epoch 4/15
11/11 [==============================] - 22s 2s/step - loss: 0.6143 - acc: 0.8022
Epoch 5/15
11/11 [==============================] - 22s 2s/step - loss: 0.4204 - acc: 0.8619
Epoch 6/15
11/11 [==============================] - 22s 2s/step - loss: 0.4184 - acc: 0.8729
Epoch 7/15
11/11 [==============================] - 22s 2s/step - loss: 0.2804 - acc: 0.9264
Epoch 8/15
11/11 [==============================] - 22s 2s/step - loss: 0.2788 - acc: 0.9146
Epoch 9/15
11/11 [==============================] - 22s 2s/step - loss: 0.2851 - acc: 0.9112
Epoch 10/15
11/11 [==============================] - 22s 2s/step - loss: 0.2094 - acc: 0.9400
Epoch 11/15
11/11 [==============================] - 22s 2s/step - lo

In [0]:
model.save_weights('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_subset/subset_3_wts.h5')


* Final model training 

In [0]:
# Loading model with no weights
model_Xcep = keras.applications.xception.Xception(weights=None,include_top=False,input_shape=(299,299,3))


In [0]:
# Freezing first 100 layers
# Around 25% of the parameters are trainable.
for layers in model_Xcep.layers[:110]:
    layers.trainable=False

# Customizing network for our dataset(12 classes)
x = model_Xcep.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='sigmoid')(x)
x = Dropout(0.5)(x)
pred = Dense(12,activation='softmax')(x)
model = Model(inputs=model_Xcep.input, outputs=pred)


In [0]:
model.load_weights('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_subset/subset_3_wts.h5')

In [0]:
nadam=keras.optimizers.nadam(lr =0.001)
model.compile(optimizer=nadam,loss='categorical_crossentropy',metrics=['accuracy'])


In [33]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    vertical_flip=False,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2)

train_generator= train_datagen.flow_from_directory(train_path,batch_size=64,target_size=(299,299),class_mode='categorical')


test_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator=test_datagen.flow_from_directory(val_path,batch_size=64,target_size=(299,299),class_mode='categorical')


Found 5760 images belonging to 12 classes.
Found 357 images belonging to 12 classes.


In [0]:
model.fit_generator(train_generator,
                    epochs=30,
                    callbacks=[mcCallBack,earlystop,decaylearn],steps_per_epoch=train_generator.samples//64,
                    validation_data = val_generator, 
    validation_steps = val_generator.samples // 64
                    )


Epoch 1/30
90/90 [==============================] - 215s 2s/step - loss: 1.1015 - acc: 0.6536 - val_loss: 1.3758 - val_acc: 0.5563
Epoch 2/30
90/90 [==============================] - 191s 2s/step - loss: 0.7493 - acc: 0.7509 - val_loss: 1.4146 - val_acc: 0.5563
Epoch 3/30
90/90 [==============================] - 199s 2s/step - loss: 0.5682 - acc: 0.8137 - val_loss: 1.5119 - val_acc: 0.6041
Epoch 4/30
90/90 [==============================] - 200s 2s/step - loss: 0.4524 - acc: 0.8521 - val_loss: 1.5439 - val_acc: 0.5904
Epoch 5/30
90/90 [==============================] - 202s 2s/step - loss: 0.3453 - acc: 0.8908 - val_loss: 1.7790 - val_acc: 0.5529
Epoch 6/30
90/90 [==============================] - 202s 2s/step - loss: 0.3062 - acc: 0.8991 - val_loss: 1.9803 - val_acc: 0.5563
Epoch 7/30
90/90 [==============================] - 198s 2s/step - loss: 0.1787 - acc: 0.9446 - val_loss: 1.7890 - val_acc: 0.6000
Epoch 8/30
90/90 [==============================] - 197s 2s/step - loss: 0.1274 - a

In [0]:
model.save_weights('/content/data/Augmented_Data/new_approach(Training_with_data_subsets)/model_subset/train_wts.h5')

In [68]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con